# Music Fingerprinting using Locality Sensitive Hashing

This notebook shows a simple system for performing retrieval of musical tracks using LSH.

Import libraries:

In [1]:
import librosa
import os
import os.path
from numpy.random import randn
from numpy import dot
import librosa.feature
import matplotlib.pyplot as plt

Set figure size:

In [2]:
plt.rcParams['figure.figsize'] = (15, 5)

Select training data:

In [3]:
training_dir = 'audio/'
training_files = [os.path.join(training_dir, f) for f in os.listdir(training_dir) if f.endswith('wav')]

Define a hash function:

In [4]:
def hash_func(vecs, projections):
    bools = dot(vecs, projections.T) > 0
    return [bool2int(bool_vec) for bool_vec in bools]

In [5]:
def bool2int(x):
    y = 0
    for i,j in enumerate(x):
        if j: y += 1<<i
    return y

In [6]:
bool2int([False, True, False, True])

10

In [7]:
X = randn(10,100)
P = randn(3,100)
hash_func(X, P)

[2, 0, 4, 0, 3, 6, 2, 0, 2, 0]

### Create three LSH structures: Table, LSH, and MusicSearch:

In [8]:
class Table:
    
    def __init__(self, hash_size, dim):
        self.table = dict()
        self.hash_size = hash_size
        self.projections = randn(self.hash_size, dim)

    def add(self, vecs, label):
        entry = {'label': label}
        hashes = hash_func(vecs, self.projections)
        for h in hashes:
            if h in self.table:
                self.table[h].append(entry)
            else:
                self.table[h] = [entry]

    def query(self, vecs):
        hashes = hash_func(vecs, self.projections)
        results = list()
        for h in hashes:
            if h in self.table:
                results.extend(self.table[h])
        return results

In [9]:
class LSH:
    
    def __init__(self, dim):
        self.num_tables = 4
        self.hash_size = 8
        self.tables = list()
        for i in range(self.num_tables):
            self.tables.append(Table(self.hash_size, dim))
    
    def add(self, vecs, label):
        for table in self.tables:
            table.add(vecs, label)
    
    def query(self, vecs):
        results = list()
        for table in self.tables:
            results.extend(table.query(vecs))
        return results

    def describe(self):
        for table in self.tables:
            print(table.table)

In [10]:
class MusicSearch:
    
    def __init__(self, training_files):
        self.frame_size = 4096
        self.hop_size = 4000
        self.fv_size = 12
        self.lsh = LSH(self.fv_size)
        self.training_files = training_files
        self.num_features_in_file = dict()
        for f in self.training_files:
            self.num_features_in_file[f] = 0
                
    def train(self):
        for filepath in self.training_files:
            x, fs = librosa.load(filepath)
            features = librosa.feature.chroma_stft(y=x, sr=fs, n_fft=self.frame_size, hop_length=self.hop_size).T
            self.lsh.add(features, filepath)
            self.num_features_in_file[filepath] += len(features)
                
    def query(self, filepath):
        x, fs = librosa.load(filepath)
        features = librosa.feature.chroma_stft(y=x, sr=fs, n_fft=self.frame_size, hop_length=self.hop_size).T
        results = self.lsh.query(features)
        print('num results', len(results))

        counts = dict()
        for r in results:
            if r['label'] in counts:
                counts[r['label']] += 1
            else:
                counts[r['label']] = 1
        for k in counts:
            counts[k] = float(counts[k])/self.num_features_in_file[k]
        return counts


Train:

In [11]:
ms = MusicSearch(training_files)
ms.train()

Test:

In [12]:
test_file = 'audio//funk_groove.mp3'
results = ms.query(test_file)

num results 3953


Display the results:

In [13]:
for r in sorted(results, key=results.get, reverse=True):
    print(r, results[r])

audio/cowbell.wav 46.0
audio/conga_groove.wav 34.16
audio/125_bounce.wav 22.372093023255815
audio/58bpm.wav 18.08695652173913
audio/classic_rock_beat.wav 13.210526315789474
audio/simple_loop.wav 13.153846153846153
audio/c_strum.wav 3.6923076923076925
audio/prelude_cmaj.wav 2.022727272727273
audio/oboe_c6.wav 1.5
audio/clarinet_c6.wav 1.1538461538461537
audio/simple_piano.wav 0.4090909090909091
